In [12]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import joblib

# --- Configurações ---
TICKER_PARA_PROCESSAR = "PETR4.SA"
# CORREÇÃO APLICADA AQUI 👇
CAMINHO_DADOS_BRUTOS = "../data/01_raw"
CAMINHO_DADOS_PROCESSADOS = "../data/03_processed"
CAMINHO_MODELOS = "../models"

# Parâmetros para o pré-processamento
PERCENTUAL_TREINO = 0.8
JANELA_DE_TEMPO = 60 # Usaremos 60 dias de histórico para prever o próximo

# Garante que os diretórios de saída existam
os.makedirs(CAMINHO_DADOS_PROCESSADOS, exist_ok=True)
os.makedirs(CAMINHO_MODELOS, exist_ok=True)

print("Configurações carregadas.")

Configurações carregadas.


In [13]:
# Constrói o caminho completo para o arquivo CSV
caminho_arquivo = os.path.join(CAMINHO_DADOS_BRUTOS, f"{TICKER_PARA_PROCESSAR}.csv")

# Carrega os dados, pulando as linhas de cabeçalho extras se necessário
# (Ajuste 'header' se os arquivos tiverem formatos diferentes)
try:
    df = pd.read_csv(
        caminho_arquivo,
        header=2, # Assumindo que o arquivo baixado pelo yfinance tem cabeçalho na primeira linha
        index_col='Date',
        parse_dates=True
    )
    print(f"Dados de {TICKER_PARA_PROCESSAR} carregados com sucesso.")
    print("Primeiras 5 linhas:")
    print(df.head())
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'")

# Tratamento de valores nulos (se houver)
    df.dropna(inplace=True)
    print("\nValores nulos removidos.")

Dados de PETR4.SA carregados com sucesso.
Primeiras 5 linhas:
            Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4   Unnamed: 5
Date                                                                   
2000-01-03    1.309279    1.309279    1.309279    1.309279  35389440000
2000-01-04    1.236851    1.236851    1.236851    1.236851  28861440000
2000-01-05    1.224371    1.224371    1.224371    1.224371  43033600000
2000-01-06    1.220137    1.220137    1.220137    1.220137  34055680000
2000-01-07    1.225708    1.225708    1.225708    1.225708  20912640000


In [14]:
print("\nNomes das colunas:")
print(df.columns)


Nomes das colunas:
Index(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], dtype='object')


In [15]:
# Seleciona a coluna 'Close' e a transforma em um array NumPy
dados_fechamento = df.iloc[:,1].values.reshape(-1, 1)

# Calcula o ponto de divisão entre treino e teste
ponto_divisao = int(len(dados_fechamento) * PERCENTUAL_TREINO)

# Separa os dados
dados_treino = dados_fechamento[:ponto_divisao]
dados_teste = dados_fechamento[ponto_divisao:]

print(f"Tamanho total dos dados: {len(dados_fechamento)}")
print(f"Tamanho do conjunto de treino: {len(dados_treino)}")
print(f"Tamanho do conjunto de teste: {len(dados_teste)}")

Tamanho total dos dados: 6449
Tamanho do conjunto de treino: 5159
Tamanho do conjunto de teste: 1290


In [16]:
# Inicializa o normalizador (scaler) para a escala de 0 a 1
scaler = MinMaxScaler(feature_range=(0, 1))

# AJUSTA o scaler APENAS com os dados de TREINO
scaler.fit(dados_treino)

# TRANSFORMA ambos os conjuntos de dados (treino e teste) com o scaler já ajustado
dados_treino_normalizados = scaler.transform(dados_treino)
dados_teste_normalizados = scaler.transform(dados_teste)

print("Dados normalizados com sucesso.")
print("Primeiros 5 valores do treino normalizado:", dados_treino_normalizados[:5].flatten())

# Salva o scaler para uso futuro (na etapa de previsão)
caminho_scaler = os.path.join(CAMINHO_MODELOS, f"{TICKER_PARA_PROCESSAR}_scaler.pkl")
joblib.dump(scaler, caminho_scaler)
print(f"\nScaler salvo em: {caminho_scaler}")

Dados normalizados com sucesso.
Primeiros 5 valores do treino normalizado: [0.02262051 0.01746596 0.0165778  0.01627647 0.01667295]

Scaler salvo em: ../models/PETR4.SA_scaler.pkl


In [17]:
def criar_sequencias(dados, janela_de_tempo):
    X, y = [], []
    for i in range(janela_de_tempo, len(dados)):
        X.append(dados[i-janela_de_tempo:i, 0])
        y.append(dados[i, 0])
    return np.array(X), np.array(y)

# Cria as sequências para os dados de treino e teste
X_train, y_train = criar_sequencias(dados_treino_normalizados, JANELA_DE_TEMPO)
X_test, y_test = criar_sequencias(dados_teste_normalizados, JANELA_DE_TEMPO)

# Ajusta o formato de X para ser [amostras, janelas, features], como o LSTM espera
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print("\nSequências criadas com sucesso.")
print("Formato de X_train:", X_train.shape)
print("Formato de y_train:", y_train.shape)
print("Formato de X_test:", X_test.shape)
print("Formato de y_test:", y_test.shape)


Sequências criadas com sucesso.
Formato de X_train: (5099, 60, 1)
Formato de y_train: (5099,)
Formato de X_test: (1230, 60, 1)
Formato de y_test: (1230,)


In [18]:
# Salva os arrays processados para não precisar refazer este notebook toda vez
np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_PROCESSAR}_X_train.npy'), X_train)
np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_PROCESSAR}_y_train.npy'), y_train)
np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_PROCESSAR}_X_test.npy'), X_test)
np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_PROCESSAR}_y_test.npy'), y_test)

print(f"\nDados processados para {TICKER_PARA_PROCESSAR} foram salvos na pasta '{CAMINHO_DADOS_PROCESSADOS}'")


Dados processados para PETR4.SA foram salvos na pasta '../data/03_processed'
